In [11]:
%matplotlib inline
import pandas as pd
from pandas.plotting import scatter_matrix
import matplotlib.pyplot as plt

In [12]:
pd.set_option('display.max_columns', 120)
pd.set_option('display.max_rows', 120)

In [13]:
def clean_and_drop(df):
    # 只篩選有包含 '住' 用途的交易案
    df = df.loc[df['Main_Usage_Living'] == 1]
    df = df.drop(columns=['Main_Usage_Living'])
    
    # 因為都是 0
    df = df.drop(columns=['Non_City_Land_Usage', 'Main_Usage_Walk', 
                          'Main_Usage_Selling',
                          'Main_Usage_SnE'])
    
    # 只有 344 筆是包含工廠用途，且都不具住宅用途，故剔除
    df = df.loc[df['Main_Usage_Manufacturing'] == 0]
    df = df.drop(columns=['Main_Usage_Manufacturing'])
    
    # 只有 76 筆是包含停車用途，且都不具住宅用途，故剔除
    df = df.loc[df['Main_Usage_Parking'] == 0]
    df = df.drop(columns=['Main_Usage_Parking'])
    
    # 只有 78 筆有農業用途，且都不具住宅用途，故剔除
    df = df.loc[df['Main_Usage_Farm'] == 0]
    df = df.drop(columns=['Main_Usage_Farm'])
    
    # NOTICE: 我沒有錢，所以我先只買 6 房以下的
    df = df.loc[df['room'] < 6]
    
    df = df.loc[df['trading_floors_count'] == 1]
    
    # 雖然有 95 個樣本包含地下室，但是樣本太少，可能不足以推廣
    # 所以先剔除，剔除完後，都是 0 所以直接 drop
    df = df.loc[df['including_basement'] == 0]
    df = df.drop(columns=['including_basement'])
    
    # 所有的樣本都不包含人行道，所以直接去除這個 feature
    df = df.drop(columns=['including_arcade'])

    # 剔除交易樓層高度是 -1 (原本有一個樣本)
    df = df.loc[df['min_floors_height'] != -1]

    # 剔除交易建物是 0 個樓層的情況
    df = df.loc[df['building_total_floors'] != 0]
    
    # 因為車位交易 50 坪以上的資料只有 22 筆，所以先去除
    # 因為浮點數在硬體儲存會有小數點，故不能直接用 == 50.0 去比較
    df = df.loc[df['Parking_Area'] < 49.5]
    
    # 把農舍，廠辦踢掉
    df = df.loc[df['Building_Types'] < 8]

    # 把超大轉移坪數刪掉
    df = df.loc[df['Transfer_Total_Ping'] < 150]
    
    # 我先刪除 area_m2, 因為覺得跟 area_ping 的意義很類似，但是不確定會不會有些微差距。
    # 因為在 future data 中，manager 都是 0，所以也把這個欄位刪除
    # trading_floor_count 有 0 的情況，這樣應該不是房屋交易
    df = df.drop(columns=['address', 'area_m2', 'manager', 'Building_Material_stone', 
                     'TDATE', 'Total_price', '編號'])
    
    # Convert the categorical features' dtype to 'category'
    category_columns = ['Type', 'Month', 'Month_raw',
                       'City_Land_Usage', 'Main_Usage_Business',
                       'Building_Material_S', 'Building_Material_R', 'Building_Material_C',
                       'Building_Material_steel', 'Building_Material_B', 
                       'Building_Material_W', 'Building_Material_iron',
                       'Building_Material_tile', 'Building_Material_clay',
                       'Building_Material_RC_reinforce',
                       'Parking_Space_Types', 'Building_Types']
    df.loc[:, category_columns] = df.loc[:, category_columns].astype('category')
    return df

In [14]:
def split_features_target(df):
    X = df.drop(columns=['Unit_Price_Ping'])
    y = df['Unit_Price_Ping']
    return X, y

def train(model, X_train, y_train):
    model.fit(X_train, y_train)
    return model

def eval(model, X_test, y_test):
    from sklearn.metrics import (r2_score, 
                                 mean_absolute_error, 
                                 mean_squared_error)
    
    y_pred = model.predict(X_test)
    r2 = r2_score(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    print(f'R2 score: {r2}')
    print(f'MAE score: {mae}')
    print(f'MSE score: {mse}')
    return r2, mae, mse

In [15]:
from sklearn.model_selection import train_test_split

In [16]:
df_future = pd.read_csv('../temp_future/output_feature/clean_data_future_train.csv')
df_future_test = pd.read_csv('../temp_future/output_feature/clean_data_future_test.csv')

In [17]:
df_future = clean_and_drop(df_future)
df_future = df_future.sample(frac=1, random_state=0)
X_train, y_train = split_features_target(df_future)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1)

In [18]:
df_future_test = clean_and_drop(df_future_test)
X_test, y_test = split_features_target(df_future_test)

In [19]:
df_future['Month'].unique()

[202109, 202112, 202004, 202110, 202005, ..., 201802, 201902, 201701, 201805, 201702]
Length: 60
Categories (60, int64): [201701, 201702, 201703, 201704, ..., 202109, 202110, 202111, 202112]

In [20]:
X_val.shape

(18274, 52)

In [ ]:
from sklearn.tree import DecisionTreeRegressor
model = train(DecisionTreeRegressor(max_depth=20), X_train, y_train)

print('Training performance: ')
eval(model, X_train, y_train)
print()
print('Evaluation performance: ')
eval(model, X_val, y_val)
print()
print('Test performance:')
eval(model, X_test, y_test)

In [21]:
from sklearn.tree import DecisionTreeRegressor
from tqdm import tqdm


avg_train_r2, avg_train_mae, avg_train_mse = 0, 0, 0
avg_val_r2, avg_val_mae, avg_val_mse = 0, 0, 0
avg_test_r2, avg_test_mae, avg_test_mse = 0, 0, 0

for i in tqdm(range(100)):
    model = train(DecisionTreeRegressor(max_depth=20), X_train, y_train)
    r2, mae, mse = eval(model, X_train, y_train)
    avg_train_r2 += r2
    avg_train_mae += mae
    avg_train_mse += mse
    
    r2, mae, mse = eval(model, X_val, y_val)
    avg_val_r2 += r2
    avg_val_mae += mae
    avg_val_mse += mse
    
    r2, mae, mse = eval(model, X_test, y_test)
    avg_test_r2 += r2
    avg_test_mae += mae
    avg_test_mse += mse

avg_train_r2 /= 100.
avg_train_mae /= 100.
avg_train_mse /= 100.

avg_val_r2 /= 100.
avg_val_mae /= 100.
avg_val_mse /= 100.

avg_test_r2 /= 100.
avg_test_mae /= 100.
avg_test_mse /= 100.

print('Training Performance:')
print(f'R2 score: {avg_train_r2}')
print(f'MAE score: {avg_train_mae}')
print(f'MSE score: {avg_train_mse}')
print()

print('Validation Performance:')
print(f'R2 score: {avg_val_r2}')
print(f'MAE score: {avg_val_mae}')
print(f'MSE score: {avg_val_mse}')
print()

print('Testing Performance')
print(f'R2 score: {avg_test_r2}')
print(f'MAE score: {avg_test_mae}')
print(f'MSE score: {avg_test_mse}')

  1%|█▏                                                                                                                         | 1/100 [00:01<02:31,  1.53s/it]

R2 score: 0.9915288934380838
MAE score: 8432.0554453398
MSE score: 263929588.0868713
R2 score: 0.9629183610035896
MAE score: 15244.809666040488
MSE score: 1118315702.3924153
R2 score: 0.8658887153405582
MAE score: 41824.62151631064
MSE score: 4983334504.37143


  2%|██▍                                                                                                                        | 2/100 [00:03<02:29,  1.52s/it]

R2 score: 0.9915288934380838
MAE score: 8432.0554453398
MSE score: 263929588.08687136
R2 score: 0.9655285179925953
MAE score: 15205.211802354821
MSE score: 1039598050.5972272
R2 score: 0.8593031542137719
MAE score: 43173.15980298978
MSE score: 5228042129.662609


  3%|███▋                                                                                                                       | 3/100 [00:04<02:27,  1.52s/it]

...

R2 score: 0.9915288929448107
MAE score: 8432.060993063315
MSE score: 263929603.45551002
R2 score: 0.9651157591455326
MAE score: 15210.254546887927
MSE score: 1052046116.8765063
R2 score: 0.8622212768222549
MAE score: 42827.40460829496
MSE score: 5119609933.820426


 99%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊ | 99/100 [02:35<00:01,  1.57s/it]

R2 score: 0.9915288929448107
MAE score: 8432.060993063315
MSE score: 263929603.45551002
R2 score: 0.9676257229658242
MAE score: 15093.507924053081
MSE score: 976350111.2889253
R2 score: 0.8697553321520157
MAE score: 42409.89959362991
MSE score: 4839657967.228049


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [02:37<00:00,  1.57s/it]

R2 score: 0.9915288934380838
MAE score: 8432.0554453398
MSE score: 263929588.0868713
R2 score: 0.9657241056693131
MAE score: 15224.525457718006
MSE score: 1033699477.1795546
R2 score: 0.8696075220269315
MAE score: 41708.6998538943
MSE score: 4845150326.0425825
Training Performance:
R2 score: 0.9915288933438552
MAE score: 8432.057088015365
MSE score: 263929591.0227559

Validation Performance:
R2 score: 0.9648797494152825
MAE score: 15217.10902802733
MSE score: 1059163746.9057972

Testing Performance
R2 score: 0.8660829173813969
MAE score: 42426.95732434642
MSE score: 4976118305.276854


In [ ]:
# from sklearn.svm import LinearSVR

# model = LinearSVR(C=1.0, epsilon=0.0, verbose=True, max_iter=300, random_state=1207)
# train(model, X_train, y_train)

# print('Training performance: ')
# eval(model, X_train, y_train)
# print()
# print('Evaluation performance: ')
# eval(model, X_test, y_test)

In [ ]:
# Too slow
# The fit time complexity is more than quadratic
# with the number of samples which makes it hard
# to scale to datasets with more than a couple of
# 10000 samples. For large datasets consider using
# LinearSVR or SGDRegressor instead, possibly
# after a Nystroem transformer.
from sklearn.svm import SVR

model = SVR(C=1.0, epsilon=0.0, verbose=True)
train(model, X_train, y_train)

print('Training performance: ')
eval(model, X_train, y_train)
print()
print('Evaluation performance: ')
eval(model, X_test, y_test)